# Here I want to try to combine the douglas data set with the parfumo dataset

### 1) Load both datasets

**import necessary libraries**

In [103]:
import pandas as pd
from rapidfuzz import process, fuzz

In [ ]:
dg = pd.read_pickle('douglas_prices_compact.pkl')
pf = pd.read_pickle('data/encoded_perfumes.pkl')

In [124]:
import polars as pl
import pickle

dg_pl = pl.DataFrame(dg)
pf_pl = pl.DataFrame(pf)

In [125]:
dg.head(3)

,brand,perfume_name,perfume_type,perfume_name_cleaned,avg_original_price_per_liter,avg_reduced_price_per_liter
0,Berdoues,Berdoues Eau de Toilette ROSE & LITCHI,Eau de Toilette,ROSE & LITCHI,329.90,NaN
1,Berdoues,Berdoues Eau de Toilette MONOI & COCO,Eau de Toilette,MONOI & COCO,349.90,NaN
2,Berdoues,Berdoues Eau de Toilette THÉ VERT & FREE,Eau de Toilette,THÉ VERT & FREE,431.67,345.33


In [126]:
dg_pl.head(3)

brand,perfume_name,perfume_type,perfume_name_cleaned,avg_original_price_per_liter,avg_reduced_price_per_liter
str,str,str,str,f64,f64
"""Berdoues""","""Berdoues Eau de Toilette ROSE …","""Eau de Toilette""","""ROSE & LITCHI""",329.9,null
"""Berdoues""","""Berdoues Eau de Toilette MONOI…","""Eau de Toilette""","""MONOI & COCO""",349.9,null
"""Berdoues""","""Berdoues Eau de Toilette THÉ V…","""Eau de Toilette""","""THÉ VERT & FREE""",431.67,345.33


In [127]:
pf.head(3)

,name,brand,year,collection,flakon_designer,perfumer,top_notes,middle_notes,base_notes,all_notes,...,ägyptisches Neroli,ägyptisches Rosengeranienblatt,ägyptisches Veilchen,ägyptisches Veilchenblatt,ägyptisches Veilchenblatt Absolue,äthiopische Myrrhe,äthiopischer Kaffee,äthiopischer Kaffee Absolue,äthiopischer Weihrauch,äthiopischer Zibet
0,Remix Cologne Anniversary Edition,4711,2017.0,None,None,Alexandra Kalle,None,None,None,[],...,0,0,0,0,0,0,0,0,0,0
1,Solarissimo - Levanzo,Azzaro,2016.0,None,None,None,"[italienische Zitrone, Passionsfrucht]","[Lavendel, Basilikum]","[Zedernholz, Patchouli]","[italienische Zitrone, Passionsfrucht, Lavende...",...,0,0,0,0,0,0,0,0,0,0
2,A*Men Ultra Zest,Mugler,2015.0,None,None,Jacques Huclier,"[Blutorange, Tangerine, Ingwer, Minze]","[Kaffee, Zimt, schwarzer Pfeffer]","[Patchouli, Vanille, Tonkabohne]","[Blutorange, Tangerine, Ingwer, Minze, Kaffee,...",...,0,0,0,0,0,0,0,0,0,0


In [128]:
pf_pl.head(3)

name,brand,year,collection,flakon_designer,perfumer,top_notes,middle_notes,base_notes,all_notes,url,Duft.rating,Duft.number_of_ratings,Haltbarkeit.rating,Haltbarkeit.number_of_ratings,Sillage.rating,Sillage.number_of_ratings,Flakon.rating,Flakon.number_of_ratings,Preis-Leistungs-Verhältnis.rating,Preis-Leistungs-Verhältnis.number_of_ratings,scent.0,scent.1,scent.2,scent.3,scent.4,scent.5,scent.6,scent.7,scent.8,scent.9,scent.10,durability.0,durability.1,durability.2,durability.3,durability.4,…,ägyptische Akazie,ägyptische Bitterorange,ägyptische Myrrhe,ägyptische Narzisse,ägyptische Orangenblüte,ägyptische Orangenblüte Absolue,ägyptische Pelargonie,ägyptische Rose,ägyptische Rosengeranie,ägyptische Rosengeranie Absolue,ägyptische Rosengeranie Orpur®,ägyptische Tagetes,ägyptische Tuberose,ägyptischer Anis,ägyptischer Jasmin,ägyptischer Jasmin Absolue,ägyptischer Jasmin-Sambac,ägyptischer Jasminum grandiflorum,ägyptischer Jasminum grandiflorum Absolue,ägyptischer Kardamom,ägyptischer Kreuzkümmel,ägyptischer Papyrus,ägyptischer Salbei,ägyptischer Seetang,ägyptisches Basilikum,ägyptisches Labdanum,ägyptisches Maiglöckchen,ägyptisches Neroli,ägyptisches Rosengeranienblatt,ägyptisches Veilchen,ägyptisches Veilchenblatt,ägyptisches Veilchenblatt Absolue,äthiopische Myrrhe,äthiopischer Kaffee,äthiopischer Kaffee Absolue,äthiopischer Weihrauch,äthiopischer Zibet
str,str,f64,str,str,str,list[str],list[str],list[str],list[str],str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Remix Cologne Anniversary Edit…","""4711""",2017.0,null,null,"""Alexandra Kalle""",null,null,null,[],"""https://www.parfumo.de/Parfums…",8.0,113.0,5.5,97.0,5.6,99.0,7.5,105.0,7.9,18.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,31.0,56.0,16.0,3.0,0.0,0.0,3.0,3.0,18.0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""Solarissimo - Levanzo""","""Azzaro""",2016.0,null,null,null,"[""italienische Zitrone"", ""Passionsfrucht""]","[""Lavendel"", ""Basilikum""]","[""Zedernholz"", ""Patchouli""]","[""italienische Zitrone"", ""Passionsfrucht"", … ""Patchouli""]","""https://www.parfumo.de/Parfums…",6.6,39.0,5.6,29.0,5.2,28.0,7.2,41.0,null,null,0.0,0.0,0.0,1.0,3.0,6.0,9.0,12.0,4.0,2.0,2.0,0.0,0.0,1.0,0.0,1.0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""A*Men Ultra Zest""","""Mugler""",2015.0,null,null,"""Jacques Huclier""","[""Blutorange"", ""Tangerine"", … ""Minze""]","[""Kaffee"", ""Zimt"", ""schwarzer Pfeffer""]","[""Patchouli"", ""Vanille"", ""Tonkabohne""]","[""Blutorange"", ""Tangerine"", … ""Tonkabohne""]","""https://www.parfumo.de/Parfums…",7.7,297.0,7.6,243.0,7.1,244.0,7.0,257.0,6.8,64.0,1.0,2.0,1.0,4.0,3.0,7.0,36.0,69.0,100.0,49.0,25.0,0.0,0.0,0.0,0.0,1.0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [129]:
pf[(pf['brand']=='Berdoues') & (pf['name']=='Rose & Lichi')]

,name,brand,year,collection,flakon_designer,perfumer,top_notes,middle_notes,base_notes,all_notes,...,ägyptisches Neroli,ägyptisches Rosengeranienblatt,ägyptisches Veilchen,ägyptisches Veilchenblatt,ägyptisches Veilchenblatt Absolue,äthiopische Myrrhe,äthiopischer Kaffee,äthiopischer Kaffee Absolue,äthiopischer Weihrauch,äthiopischer Zibet


##### Fuzzy matching

Finding the perfumes with the exact names is hard, because they are not writting 100% the same, so we want to find the match that has the highest fuzzy score

Let's take a look at how many perfumes are written the same before doing fuzzy adjustments

In [143]:
pf[pf['brand'].isna()]

,name,brand,year,collection,flakon_designer,perfumer,top_notes,middle_notes,base_notes,all_notes,...,ägyptisches Neroli,ägyptisches Rosengeranienblatt,ägyptisches Veilchen,ägyptisches Veilchenblatt,ägyptisches Veilchenblatt Absolue,äthiopische Myrrhe,äthiopischer Kaffee,äthiopischer Kaffee Absolue,äthiopischer Weihrauch,äthiopischer Zibet
4003,NaN,NaN,NaN,NaN,NaN,NaN,"[Damaszener-Rose, fruchtige Noten, Bergamotte,...","[blumige Noten, Cypriol, Akigalawood®, Benzoe,...","[Leder, Oud, Weihrauch, Amber, Bibergeil, Eich...","[Damaszener-Rose, fruchtige Noten, Bergamotte,...",...,0,0,0,0,0,0,0,0,0,0
4021,NaN,NaN,NaN,NaN,NaN,NaN,"[Schokolade, Vanille]",[weiße Blüten],"[Tonkabohne, Moschus]","[Schokolade, Vanille, weiße Blüten, Tonkabohne...",...,0,0,0,0,0,0,0,0,0,0
4029,NaN,NaN,NaN,NaN,NaN,NaN,"[Ozon, rosa Pfeffer, aquatische Noten, grüne N...","[Gewürze, bulgarische Rose, weiße Blüten, Ylan...","[Sandelholz, Vanille, Ambra, schwarzes Meersal...","[Ozon, rosa Pfeffer, aquatische Noten, grüne N...",...,0,0,0,0,0,0,0,0,0,0
4040,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,[],...,0,0,0,0,0,0,0,0,0,0
4047,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,[],...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13883,NaN,NaN,NaN,NaN,NaN,NaN,"[Bergamotte, Gewürzstrauch, Ylang-Ylang, Jasmin]","[Pfirsich, Unterholz]","[Vetiver, Sandelholz]","[Bergamotte, Gewürzstrauch, Ylang-Ylang, Jasmi...",...,0,0,0,0,0,0,0,0,0,0
13885,NaN,NaN,NaN,NaN,NaN,NaN,"[Feige, Strohblume, Jasmin, Mandarine]","[Kamille, Meerlilie, Zistrose]","[Wacholderholz, Zedernholz, Feigenholz]","[Feige, Strohblume, Jasmin, Mandarine, Kamille...",...,0,0,0,0,0,0,0,0,0,0
13928,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,[],...,0,0,0,0,0,0,0,0,0,0
13952,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,[],...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pf_brands = set(pf["brand"].dropna())

In [139]:
dg_brands = set(dg['brand'])
pf_brands = set(pf['brand'])
pf_brands = set(pf["brand"].dropna())

print(f'{len(dg_brands)} brands in douglas dataframe')
print(f'{len(pf_brands)} brands in parfumo dataframe')

common_brands = dg_brands & pf_brands

print(f'{len(common_brands)} brands are common')

380 brands in douglas dataframe
782 brands in parfumo dataframe
192 brands are common


In [173]:
# Get unique brands as sets
dg_brands = set(dg_pl["brand"].drop_nulls().to_list())
pf_brands = set(pf_pl["brand"].drop_nulls().to_list())

# Print the counts
print(f"{len(dg_brands)} brands in Douglas dataframe")
print(f"{len(pf_brands)} brands in Parfumo dataframe")

# Find common brands
common_brands = dg_brands & pf_brands
only_douglas = dg_brands - pf_brands   # Brands unique to Douglas
only_parfumo = pf_brands - dg_brands   # Brands unique to Parfumo

print(f"{len(common_brands)} brands are common")
print(f"{len(only_douglas)} only in douglas")
print(f"{len(only_parfumo)} only in parfumo")


379 brands in Douglas dataframe
782 brands in Parfumo dataframe
242 brands are common
137 only in douglas
540 only in parfumo


In [158]:
# sort and compare to see what they look like
only_douglas = sorted(only_douglas)
only_parfumo = sorted(only_parfumo)

print(f'douglas: {only_douglas[:10]}')
print(f'parfumo: {only_parfumo[:10]}')

douglas: ['4711 Acqua Colonia', 'ACCENDIS', 'ANFAS', 'ARMAF', 'ASABI', 'Above Average Humanity', 'Absolument Parfumeur', 'Acqua Alpes', 'Aether', 'Afnan']
parfumo: ['1969', '4160 Tuesdays', 'Aaron Terence Hughes', 'Abdul Samad Al Qurashi', 'Acqua dellElba', 'Acqua di Stresa', 'Adam Bensahili', 'Adam Levine', 'Adi ale Van', 'Adidas']


In [161]:
# Example lists
dg_brands = only_douglas
pf_brands = only_parfumo

# Convert lists to sets to remove duplicates
dg_brands_set = set(dg_brands)
pf_brands_set = set(pf_brands)

# Get a globally sorted list of unique brand names
all_brands = sorted(dg_brands_set | pf_brands_set)  # Union of sets to merge & sort

# Create columns by checking if each brand exists in the respective set
df = pl.DataFrame({
    "Douglas Brands": [brand if brand in dg_brands_set else None for brand in all_brands],
    "Parfumo Brands": [brand if brand in pf_brands_set else None for brand in all_brands]
})

print(df)

shape: (778, 2)
┌────────────────────┬────────────────┐
│ Douglas Brands     ┆ Parfumo Brands │
│ ---                ┆ ---            │
│ str                ┆ str            │
╞════════════════════╪════════════════╡
│ null               ┆ 1969           │
│ null               ┆ 4160 Tuesdays  │
│ 4711 Acqua Colonia ┆ null           │
│ ACCENDIS           ┆ null           │
│ ANFAS              ┆ null           │
│ …                  ┆ …              │
│ oLiente            ┆ null           │
│ null               ┆ olakala        │
│ perfume.sucks      ┆ null           │
│ null               ┆ pernoire       │
│ null               ┆ soOud          │
└────────────────────┴────────────────┘


In [162]:
df.write_excel("brands_differences_dg_pf.xlsx")

In [169]:
brands_pd = pd.read_excel("brands_differences_dg_pf.xlsx", sheet_name="Sheet1")

# Convert to Polars DataFrame
brands_pl = pl.from_pandas(brands_pd)


In [171]:
df_filtered = brands_pl.filter(
    brands_pl["Douglas Brands"].is_not_null() & brands_pl["Parfumo Brands"].is_not_null()
)

print(df_filtered)

shape: (53, 2)
┌─────────────────────────────┬────────────────────────┐
│ Douglas Brands              ┆ Parfumo Brands         │
│ ---                         ┆ ---                    │
│ str                         ┆ str                    │
╞═════════════════════════════╪════════════════════════╡
│ Afnan                       ┆ Afnan Perfumes         │
│ Armani                      ┆ Giorgio Armani         │
│ BURBERRY                    ┆ Burberry               │
│ BVLGARI                     ┆ Bvlgari                │
│ BYREDO                      ┆ Byredo                 │
│ CALVIN KLEIN                ┆ Calvin Klein           │
│ CARON Paris                 ┆ Caron                  │
│ CHANEL                      ┆ Chanel                 │
│ COACH                       ┆ Coach                  │
│ CORBO                       ┆ Corbo                  │
│ DIOR                        ┆ Dior                   │
│ DKNY                        ┆ DKNY  Donna Karan      │
│ EIGHT & BOB   

In [172]:
# Create a mapping dictionary from Douglas to Parfumo brand names
brand_mapping = dict(zip(df_filtered["Douglas Brands"], df_filtered["Parfumo Brands"]))

# Replace values in the 'brand' column using the mapping
dg_pl = dg_pl.with_columns(
    dg_pl["brand"].replace(brand_mapping).alias("brand")
)

print(dg_pl)

shape: (4_501, 6)
┌──────────┬─────────────────┬─────────────────┬─────────────────┬────────────────┬────────────────┐
│ brand    ┆ perfume_name    ┆ perfume_type    ┆ perfume_name_cl ┆ avg_original_p ┆ avg_reduced_pr │
│ ---      ┆ ---             ┆ ---             ┆ eaned           ┆ rice_per_liter ┆ ice_per_liter  │
│ str      ┆ str             ┆ str             ┆ ---             ┆ ---            ┆ ---            │
│          ┆                 ┆                 ┆ str             ┆ f64            ┆ f64            │
╞══════════╪═════════════════╪═════════════════╪═════════════════╪════════════════╪════════════════╡
│ Berdoues ┆ Berdoues Eau de ┆ Eau de Toilette ┆ ROSE & LITCHI   ┆ 329.9          ┆ null           │
│          ┆ Toilette ROSE … ┆                 ┆                 ┆                ┆                │
│ Berdoues ┆ Berdoues Eau de ┆ Eau de Toilette ┆ MONOI & COCO    ┆ 349.9          ┆ null           │
│          ┆ Toilette MONOI… ┆                 ┆                 ┆       

In [174]:
dg_pl.write_csv("douglas_prices_compact.csv")

Trying to get the dougals datframe **_dg_** to have the same brand names as the original

In [132]:
def match_brand(brand, brand_list, threshold=85):
    match, score, _ = process.extractOne(brand, brand_list, scorer=fuzz.ratio)
    return match if score >= threshold else brand  # Keep original if no good match

# Apply fuzzy matching
dg['brand'] = dg['brand'].apply(lambda x: match_brand(x, pf['brand'].unique()))

# Display updated dg dataframe
print(dg)

         brand                                       perfume_name  \
0     Berdoues             Berdoues Eau de Toilette ROSE & LITCHI   
1     Berdoues              Berdoues Eau de Toilette MONOI & COCO   
2     Berdoues           Berdoues Eau de Toilette THÉ VERT & FREE   
3     Berdoues       Berdoues Eau de Parfum Verveine & Clémentine   
4     Berdoues  Berdoues Eau de Parfum Collection Grand Crus G...   
...        ...                                                ...   
5882     Creed  Creed Eau de Parfum Millesime for Women Love I...   
5883     Creed  Creed Eau de Parfum Millesime for Men Original...   
5884     Creed      Creed Parfum Millesime for Women Wind Flowers   
5885     Creed  Creed Eau de Parfum Acqua Originale Citrus Big...   
5886     Creed  Creed Eau de Parfum Acqua Originale Zeste Mand...   

         perfume_type                          perfume_name_cleaned  \
0     Eau de Toilette                                 ROSE & LITCHI   
1     Eau de Toilette        

In [110]:
dg_brands = set(dg['brand'])
pf_brands = set(pf['brand'])

print(f'{len(dg_brands)} brands in douglas dataframe')
print(f'{len(pf_brands)} brands in parfumo dataframe')

common_brands = dg_brands & pf_brands

print(f'{len(common_brands)} brands are common')

380 brands in douglas dataframe
1487 brands in parfumo dataframe
192 brands are common


In [111]:
# add hf dataset

hf = pd.read_json('data/perfumes_huggingface.json')

In [112]:
hf.head()

,brand,name_perfume,family,subfamily,fragrances,ingredients,origin,gender,years,description,image_name
0,Fiorucci,Wallstreet,FLORAL,AMBERY (ORIENTAL),Floral Amber Fresher,"[Jasmine, Lemon, Spicy Notes, Gardenia, Rose, ...",Brazil,Male,2015,"A modern aroma, ideal for urban men and that h...",jsunc6exf3hcx3bfbnx0jw5f2s4zske1770lzwplo5hn2v...
1,Givenchy,Désinvolte,FLORAL,FLORAL,Floral Fresher White Flower,"[Orange Blossom, Jasmine (Grandiflorum), Vetiv...",France,Unisex,2021,Désinvolte is a generous bouquet of white flow...,iui51coumnkhypsmwlvlswjeo3qdst1mee7xanig3zq9xx...
2,Issey Miyake,L'Eau D'Issey Shade Of Sunrise 2019,FLORAL,FLORAL,Floral Fresher White Flower,"[Jasmine, Ylang-ylang, Heliotrope, Osmanthus, ...",Japan,Female,2019,"Day 1, 5:45 am. "" evokes a joyous, exciting an...",ufrzbh341nph8c35z1actgcufh53y134hrnun1jthvtktt...
3,Guess,Guess Night,AROMATIC FOUGERE,AROMATIC FOUGERE,Aromatic Fougère Fresher Aromatic,"[Geranium, Chilipepper, Patchouli (Indonesia),...",United States of America,Male,2013,Confidence owns the night. A sexy energy that ...,meyc1xid3cdr4nwqmgmjocw84s57te73kpo5hdhnntuj1j...
4,Tfk,Signature Line : Nº 32,FLORAL,CITRUS,Floral Crisp Citrus Fruity,"[Bergamot, Iris / Orris, Jasmine, Neroli, Rose...",None,Unisex,2015,"32 is a rich perfume, enhanced by a bouquet of...",4x263jhomb3kilrjxl49figkykzrtvxinqk7djko17cvse...


In [113]:
hf[hf['brand']=='Xerjoff']

,brand,name_perfume,family,subfamily,fragrances,ingredients,origin,gender,years,description,image_name
233,Xerjoff,Comandante!,FLORAL,ALDEHYDIC,Soft Floral Classical,"[Floral Notes, Tobacco Leaf]",Italy,Unisex,2013,Comandante! by Xerjoff Join the Club is a perf...,eftwos8ek6drpp9sz1o9noefgd97chkigh9y2vxeyg2np6...
354,Xerjoff,Damarose,FLORAL,FLORAL,Floral Classical Rose,"[Rose (Bulgaria), Bergamot, Lily Of The Valley...",Italy,Unisex,2010,"Damarose is a classic rose chypre, a type of r...",d2n9xu1vhxwq6h2vwjuix8nu69t43ochpyae880fkat9gm...
701,Xerjoff,Zanzibar,WOODY,AMBERY (ORIENTAL),Woody Amber Fresher Fruity,"[Cedarwood, Osmanthus, Patchouli, Fruity Notes]",Italy,Unisex,2014,"Zanzibar is an enigma, like a mirage in the de...",vzmtytmmu4dgatophzhtic51x8inn7uhh8dgj0yezrpjjg...
823,Xerjoff,Shingl,FLORAL,MUSK SKIN,Soft Floral Fresher Citrus,"[Iris / Orris, Musk, Bergamot, Amber, Anise, P...",Italy,Unisex,2008,Shingl is evocative of a stylish garden party ...,b6kkkyh0rwd0buoiwpeye8o1rjfq904pprqaxghuctips6...
832,Xerjoff,Alexandria Ii Anniversary,WOODY,AMBERY (ORIENTAL),Woody Amber Classical,"[Oud / Agarwood, Musk, Cypriol / Nagarmota, Am...",Italy,Unisex,2024,Alexandria II Anniversary is emblematic of the...,b7eyt80wwbg1m04tx06cwlbcukfww5tne0xhmtq9ssla0q...
...,...,...,...,...,...,...,...,...,...,...,...
25146,Xerjoff,Amphorae 39 2022,AMBERY (ORIENTAL),CITRUS,Amber Crisp Citrus,"[Bergamot, Musk, Nutmeg, Vanilla (Madagascar),...",None,Unisex,2022,Amphorae 39 is a rare combination of aged musk...,vn01r93roafk1mzg9mj6ram8d6i7funiyq5jyrymhi938t...
25236,Xerjoff,Kemi,WOODY,AMBERY (ORIENTAL),Woody Amber Classical,"[Oud (Laos), Caramel, Vanilla, Cedarwood, Sand...",Italy,Unisex,2014,Taking its name from the ancient word for Egyp...,6rc9dpfdovcuxjycgew8abgyb65h2c5uq9bp8vmqvhurnz...
25404,Xerjoff,Casafutura,WOODY,WOODY,Woods Richer,"[Sandalwood, Rose, Rosemary, Cedarwood, Gerani...",Italy,Unisex,2021,Casamorati celebrates Futurism with Casafutura...,gi3ik4hj9xsye6zii345m8iilysz5ub8l1mc3d9vuw4r4w...
25606,Xerjoff,Oud Luban,WOODY,AMBERY (ORIENTAL),Woody Amber Richer,"[Oud / Agarwood, Rose (Bulgaria), Cedarwood, V...",Italy,Unisex,2015,Inspired by the ancient Arabian art of perfume...,6oxzz9ddv8sx6wiyagw8afsl90w3tu56r340q9hjiabceg...


In [114]:
from rapidfuzz.process import extractOne

from rapidfuzz.distance import Levenshtein

from rapidfuzz.fuzz import ratio

from rapidfuzz import fuzz

In [115]:
reference = 'Tobacco Vanille'
liste = ['Tobacco Vanill', 'Tobacco Vanille Eau de Parfum', 'Tom Ford Tobacco Vanille Eau de Parfum', 'TOBACCO VANILLE']

In [116]:
extractOne(reference, liste, scorer=ratio)

('Tobacco Vanill', 96.55172413793103, 0)

In [117]:
extractOne(reference, liste, scorer=Levenshtein.normalized_similarity)

('Tobacco Vanill', 0.9333333333333333, 0)

In [118]:
for el in liste:
    print(fuzz.ratio(reference, el))

96.55172413793103
68.18181818181819
56.60377358490566
19.999999999999996
